# Tutorial on linear algebra in NumPy and Jax

## TOC:
* [Basics](#basics)
* [Sparse matrices](#sparse)
* [Broadcasting](#broadcasting)
* [Einstein summation](#einstein)
* [Norms](#size)
* [Eigenvalue decomposition](#EVD)
* [Singular value decomposition](#SVD)
* [Other decompositions](#decomp)
* [Matrix calculus](#calculus)
* [Linear systems of equations](#linear)

In [64]:
import jax.numpy as np
import numpy as onp # original numpy 
onp.set_printoptions(precision=3)

## Basics <a class="anchor" id="basics"></a>

In [3]:
# Create 1d vector
v = np.array([0,1,2]) # 1d vector
print(v.ndim) ## 1
print(v.shape) ## (3,)


# Note that Python uses 0-indexing, not 1-indexing.
# Thus the elements are accessed as follows:
print(v[0], v[1], v[2]) ## 0 1 2

[0 1 2]
1
(3,)
0 1 2


In [66]:
# Create 2d array
A = np.array([ [0,1,2], [3,4,5] ]) 
print(A)
## [[0, 1, 2],
##  [3, 4, 5]])
print(A.ndim) ## 2
print(A.shape) ## (2,3)
print(A.size) ## 6
print(A.T.shape) ## (3,2)

[[0 1 2]
 [3 4 5]]
2
(2, 3)
6
(3, 2)


In [67]:
# If we want to make a vector into a matrix with one row, we can use any of the following:

x = np.array([1,2]) # vector
X1 = np.array([x]) # matrix with one row
X2 = np.reshape(x, (1,-1))
X3 = x[None, :]
X4 = x[np.newaxis, :]
assert np.array_equal(X1, X2)
assert np.array_equal(X1, X3)
print(np.shape(X1)) ## (1,2)



# If we want to make a vector into a matrix with one column, we can use any of the following:
x = np.array([1,2]) # vector
X1 = np.array([x]).T # matrix with one column
X2 = np.reshape(x, (-1,1))
X3 = x[:, None]
X4 = x[:, np.newaxis]
assert np.array_equal(X1, X2)
assert np.array_equal(X1, X3)
print(np.shape(X1)) ## (2,1)

(1, 2)
(2, 1)


In [68]:
# Here is how to create a one-hot encoding of integers.

def one_hot(x, k, dtype=np.float32):
  return np.array(x[:, None] == np.arange(k), dtype)


# Example
x = np.array([1,2,0,2]);
X = one_hot(x, 3)
print(X)



[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [69]:

# We can construct arrays from a list of column vectors as follows:
A1 = np.array([ [0,1,2], [3,4,5] ]) 
col0 = A1[:,0]; col1 = A1[:,1]; col2=A1[:,2];
A2 = np.stack([col0,col1,col2],axis=1)
assert np.array_equal(A1, A2)

# We can construct arrays from a list of row vectors as follows:
row0=A1[0,:]; row1=A1[1,:];
A2 = np.stack([row0,row1],axis=0)
assert np.array_equal(A1, A2)

In [71]:
# We can construct arrays from a list of arrays
# using the hstack or vstack functions,
# which stack horizontally or vertically,  as illustrated below.

M = np.array([[9,8,7],[6,5,4]])
C = np.array([[99], [99]])
A1 = np.concatenate([M, C], axis=1)
A2 = np.hstack([M, C])
#A3 = np.c_[M, C] # c_ does not work in jax
assert np.array_equal(A1, A2)
#assert np.array_equal(A1, A3)
print(A1)



[[ 9  8  7 99]
 [ 6  5  4 99]]


In [72]:

R = np.array([[1,2,3]])
A1 = np.concatenate([R, M], axis=0)
A2 = np.vstack([R, M])
assert np.array_equal(A1, A2)
print(A1)


[[1 2 3]
 [9 8 7]
 [6 5 4]]


In [73]:
# A very common idiom  is to add a column of 1s to a datamatrix.
# We can do this using horizontal stacking (along the columns) as follows.

X = np.array([[9,8,7],[6,5,4]])
N = np.shape(X)[0] # num. rows
X1 = np.hstack([np.ones((N,1)), X])
print(X1)

[[1. 9. 8. 7.]
 [1. 6. 5. 4.]]


In [74]:

# We can flatten a matrix to a vector (concatenating its rows, one by one) using ravel

A = np.reshape(np.arange(6),(2,3))
print(A.ravel()) ##  [0 1 2 3 4 5]


[0 1 2 3 4 5]


In numpy,  arrays are layed out in memory
such that, if we iterate over neighboring elements,
the rightmost index changes the fastest.
This is called row-major order,
and is used by other languages such as C++, Eigen and PyTorch.
By contrast, other languages (such as Julia, Matlab, R and Fortran)
use column-major order.
See below for an illustration of the difference.


<table border="0" width="400">
    <td><img src="figures/rowMajor.png"  width="200"></td>
    <td><img src="figures/colMajor.png"  width="200"></td>
</table>


(Source: https://commons.wikimedia.org/wiki/File:Row_and_column_major_order.svg)

Thus in numpy, for speed reasons, we should always write loops like this:
```
A = np.reshape(np.arange(6),(2,3))
d1, d2 = np.shape(A)
for i in range(d1):
  for j in range(d2):
    # Do something with A[i,j]
 ```

For similar reasons, data matrices are usually stored
in the form $(N,D)$, where $N$ is the batchsize (number of examples),
so that we can efficiently extract minibatches by slicing blocks of consecutive memory.

In [75]:
## We can create a tensor in numpy as in this example:

T = np.ndarray([2,3,4]) # fill with random values
T = np.reshape(np.arange(24),(2,3,4)) # fill with 0..23
print(np.shape(T))
print(T)



(2, 3, 4)
[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


In [76]:
#We can permute the order of the dimensions using np.transpose.

x = np.ones((1, 2, 3))
print(np.transpose(x, (1, 0, 2)).shape) ## (2, 1, 3)

#Note that this does not actually move the data in memory
#(which would be slow),
#it merely provides a different \keywordDef{view} of the same data,
#i.e., it changes the mapping from $n$-dimensional vectors of
#subscripts to 1d integers.

(2, 1, 3)


In [81]:
# matrix multiplication 

A = onp.random.rand(2,3);
B = onp.random.rand(3,4);
C = np.dot(A,B)
assert np.shape(C) == (2,4)
print(C)
C2 = A.dot(B)
C3 = A @ B
assert np.allclose(C, C2)
assert np.allclose(C, C3)

#Note that we need to use np.dot(A,B)
#if we use A * B, Python tries to compute the elementwise product,
#which is invalid, since $A$ and $B$ have incompatible shapes.


[[1.298 0.859 1.059 0.369]
 [1.568 0.942 1.311 0.477]]


In [82]:
# Outer products

x = np.arange(1,3); y = np.arange(1,3); 
A = np.outer(x,y);
print(A)


[[1 2]
 [2 4]]


In [83]:
# We can sum across the rows

X = np.reshape(np.arange(6), (2,3))
XS = np.dot(np.ones((1,2)), X)
print(XS)
XS2 = np.sum(X, axis=0)
assert np.allclose(XS, XS2)


[[3. 5. 7.]]


In [84]:
# We can sum across the columns 

X = np.reshape(np.arange(6), (2,3))
XS = np.dot(X, np.ones((3,1)))
print(XS)
XS2 = np.sum(X, axis=1).reshape(-1, 1)
assert np.allclose(XS, XS2)


[[ 3.]
 [12.]]


In [85]:
# We can sum across all entries

X = np.reshape(np.arange(6), (2,3))
S1 = np.dot(np.ones((1,2)), np.dot(X, np.ones((3,1))))[0]
S2 = np.sum(X)
assert np.allclose(S1, S2)


In [86]:
# Kronecker product

np.kron(np.eye(2), np.ones((2,2)))


DeviceArray([[1., 1., 0., 0.],
             [1., 1., 0., 0.],
             [0., 0., 1., 1.],
             [0., 0., 1., 1.]], dtype=float32)

In [99]:
# Vector Norms
x = np.arange(6)
print(np.linalg.norm(x, 2) ** 2)
print(np.sum(np.power(x, 2)))
print(np.linalg.norm(x, np.inf))

# Matrix norms
A = onp.random.randn(4,4)
print(np.linalg.norm(A, ord=2))
print(np.linalg.norm(A, ord='nuc'))
print(np.linalg.norm(A, ord='fro'))



7.4161983
5.0
3.7022145
7.0724216
4.320558
2.8176277
2.5269188897052897
12.36230186647291


In [ ]:
# Size of a matrix - not  all operations are supported by jax

print(np.trace(A))
print(onp.linalg.det(A))
print(onp.linalg.cond(A))

## Sparse matrices  <a class="anchor" id="sparse"></a>

In [87]:
from scipy.sparse import diags
A = diags([1,2,3])
print(A)
print(A.toarray())


  (0, 0)	1.0
  (1, 1)	2.0
  (2, 2)	3.0
[[1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 3.]]


In [88]:
# Block diagonal

from scipy.linalg import block_diag
block_diag([2, 3], [[4, 5], [6, 7]])



array([[2, 3, 0, 0],
       [0, 0, 4, 5],
       [0, 0, 6, 7]])

Band diagonal

See (https://pypi.org/project/bandmat)

## Broadcasting  <a class="anchor" id="broadcasting"></a>

In numpy, the command A * B computes the elementwise multiplication of arrays or tensors A and B.
If these arrays have different shapes,
they will be automatically converted to have compatible shapes by
implictly replicating  certain dimensions; this is called
**broadcasting**. The following conversion rules are applied
in order:

* If the two arrays differ in their number of dimensions, the
   shape of the one with fewer dimensions is padded with ones on the
   left side. For example, a scalar will be converted to a vector,
   and a vector to a matrix with one row.
* If the shape of the two arrays does not match in any dimension,
   the array with shape equal to 1 in that dimension is stretched to
   match the other shape, by replicating the corresponding contents.
* If in any dimension the sizes disagree and neither is equal to
   1, an error is raised.

This is illustrated below.

![Broadcasting](figures/broadcasting.png "Broadcasting")


Source: Figure 2.4 of [VanderPlas2016](https://github.com/jakevdp/PythonDataScienceHandbook).
Thanks to Jake VanderPlas.
Made by [broadcasting_fig.py](https://github.com/probml/pyprobml/blob/master/scripts/broadcasting_fig.py)


In [89]:
# Example: scaling each column
X = np.reshape(np.arange(6), (2,3))
s = np.array([1,2,3])
XS = X * s 
print(XS)
XS2 = np.dot(X, np.diag(s)) # post-multiply by diagonal
assert np.allclose(XS, XS2)


[[ 0  2  6]
 [ 3  8 15]]


In [90]:
# Example: scaling each row
X = np.reshape(np.arange(6), (2,3))
s  = np.array([1,2])
XS = X *  np.reshape(s, (-1,1)) 
print(XS)
XS2 = np.dot(np.diag(s), X) # pre-multiply by diagonal
assert np.allclose(XS, XS2)

[[ 0  1  2]
 [ 6  8 10]]


## Einstein summation  <a class="anchor" id="broadcasting"></a>

Einstein summation lets us write formula such as  inputs -> outputs, which name the dimensions 
of the input tensor and output tensors; dimensions which are not named in the output are summed over - this is called **tensor contraction**.


In [91]:
# Sample data
a = np.arange(3)
b = np.arange(3)
A = np.arange(6).reshape(2,3)
B = np.arange(15).reshape(3,5)
S = np.arange(9).reshape(3,3)
T = onp.random.randn(2,2,2,2)

Now consider einsum with  a single tensor.

In [92]:


# Matrix transpose
assert np.allclose(A.T, np.einsum('ij->ji', A))

# Sum all elements
assert np.allclose(np.sum(A), np.einsum('ij->', A))

# Sum across rows
assert np.allclose(np.sum(A, axis=0), np.einsum('ij->j', A))

# Sum across columns
assert np.allclose(np.sum(A, axis=1), np.einsum('ij->i', A))

# Sum specific axis of tensor
assert np.allclose(np.sum(T, axis=1), np.einsum('ijkl->ikl', T))
assert np.allclose(np.sum(np.sum(T, axis=0), axis=0), np.einsum('ijkl->kl', T))

# repeated indices with one arg extracts diagonals
assert np.allclose(np.diag(S), np.einsum('ii->i', S))
          
# Trace
assert np.allclose(np.trace(S), np.einsum('ii->', S))


Now consider einsum with 2 tensors.

In [93]:


# Matrix vector multiplication
assert np.allclose(np.dot(A, b), np.einsum('ik,k->i', A, b))

# Matrix matrix multiplication
assert np.allclose(np.dot(A, B), np.einsum('ik,kj->ij', A, B))
assert np.allclose(np.matmul(A, B), np.einsum('ik,kj->ij', A, B))

# Inner product 
assert np.allclose(np.dot(a, b), np.einsum('i,i->', a, b))
assert np.allclose(np.inner(a, b), np.einsum('i,i->', a, b))

# Outer product
assert np.allclose(np.outer(a, b), np.einsum('i,j->ij', a, b))

# Elementwise product
assert np.allclose(a * a, np.einsum('i,i->i', a, a))
assert np.allclose(A * A, np.einsum('ij,ij->ij', A, A))
assert np.allclose(np.multiply(A, A), np.einsum('ij,ij->ij', A, A))


 As a more complex example,
 suppose we have a 3d tensor $S_{ntk}$ where $n$ indexes examples in the
 batch, $t$ indexes locations in the sequence, and $k$ indexes words
 in a one-hot representation.
 Let $W_{kd}$ be an embedding matrix that maps sparse one-hot vectors
 $R^k$  to dense vectors in $R^d$.
 We can convert the batch of sequences of one-hots
 to a batch of sequences of embeddings as follows:
$$
E_{ntd} = \sum_k S_{ntk} W_{kd}
$$
We can compute the sum of the embedding vectors for
each sequence (to get a global representation
of each bag of words) as follows:
$$
E_{nd} = \sum_k \sum_t S_{ntk} W_{kd}
$$
Finally we can pass each sequence's vector representation
through another linear transform $V_{dc}$ to map to the logits over a
classifier
with $c$ labels:
$$
L_{nc} = \sum_d E_{nd} V_{dc}
= \sum_d \sum_k \sum_t S_{ntk} W_{kd} V_{dc}
$$
In einsum notation, we have
$$
L_{nc} = S_{ntk} W_{kd} V_{dc}
$$
We sum  over $k$ and $d$  because those
indices occur twice on the RHS.
We sum over $t$  because that index does not occur
on the LHS.

In [95]:
# sentence embedding example in code

N = 2; C = 3; D = 4; K = 5; T = 6;
S = onp.random.randn(N, T, K)
W = onp.random.randn(K, D)
V = onp.random.randn(D, C)
Lfast = np.einsum('ntk,kd,dc->nc', S, W, V)
# Compare to brute force way of computing L below.
# We can only do elementwise assignment to L in original numpy, not jax
L = onp.zeros((N,C))
for n in range(N):
    for c in range(C):
        s = 0
        for d in range(D):
            for k in range(K):
                for t in range(T):
                    s += S[n,t,k] * W[k,d] * V[d,c]
        L[n,c] = s # does not work in jax
assert np.allclose(L, Lfast)

In [96]:
# Optimization

path = np.einsum_path('ntk,kd,dc->nc', S, W, V, optimize='optimal')[0]
assert np.allclose(L, np.einsum('ntk,kd,dc->nc', S, W, V, optimize=path))


## Eigenvalue decomposition (EVD)<a class="anchor" id="EVD"></a>

In [63]:
M = np.random.randn(3, 4)
A = np.dot(M, M.T) # ensure A is positive definite
evals, evecs = np.linalg.eig(A)
print(evals)
print(evecs)

# Sort columns so one with largest eval is first
idx = np.argsort(np.abs(evals))[::-1] # largest first
evecs = evecs[:, idx] # sort columns
evals = evals[idx]
print(evals)
print(evecs)

[6.194 0.091 1.052]
[[-0.118  0.919 -0.376]
 [ 0.584 -0.242 -0.775]
 [ 0.803  0.311  0.508]]
[6.194 1.052 0.091]
[[-0.118 -0.376  0.919]
 [ 0.584 -0.775 -0.242]
 [ 0.803  0.508  0.311]]
